In [1]:
import json
import pickle
import time
from tqdm import tqdm

valfile = "/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/downstream/val/val.jsonl"
def load_jsonl(path):
    with open(path, 'r') as f:
        a = f.readlines()
        g = [json.loads(i) for i in a]
    return g
val_ds = load_jsonl(valfile)
doping_test = load_jsonl('/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/doping_test.jsonl')
mof1_test = load_jsonl('/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/mof1_test.jsonl')
mof2_test = load_jsonl('/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/mof2_test.jsonl')
with open("/home/cse/btech/cs1200389/MatLlama/MatLLaMA/datasets/ift/final_ift/discomat_dense_test.jsonl", 'r') as f:
    discomat_test = [json.loads(line) for line in f.readlines()]


In [ ]:
import os
# os.environ['http_proxy'] ="10.10.78.22:3128"
# os.environ['https_proxy'] ="10.10.78.22:3128"
import openai
from openai import OpenAI

client = OpenAI(
  api_key = "YOUR API KEY"
  # organization='IITD',
  # project='',
)

In [4]:
model_name = "gpt-4"
def get_prompt(input_dict):
    system_prompt = input_dict['system']
    prompt = input_dict['question']
    return system_prompt, prompt

def get_output(input_dict, additional = ""):
    sys, prompt = get_prompt(input_dict)
    sys = sys + additional
    messages = [
        {"role": "system", "content": sys},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=model_name,  
        messages=messages,
        temperature=0.0,
        max_tokens=2000,
        frequency_penalty=0.0
    )
    response_text = response.choices[0].message.content
    return response_text

outname = model_name + "_val"
print(outname)
print(get_output(val_ds[19], additional = ". Do not output any additional text. and give proper parseable json format when asked for it"))

gpt-4_val


APIConnectionError: Connection error.

## Downstream run

In [4]:
# downstream_outs = sorted(downstream_outputs, key = lambda x : x[0])

In [5]:
# actual = [downstream_outs[0]]
# for i in range(1,len(downstream_outs)):
#     if(downstream_outs[i-1][0] == downstream_outs[i][0]):
#         actual[-1] = downstream_outs[i]
#     else:
#         actual.append(downstream_outs[i])

In [6]:
import pickle
import time
from tqdm import tqdm
import threading 
from threading import Thread
downstream_outputs = {}
def get_downstream_outputs_range(start_idx, end_idx):
    exceptions = []
    start_time = time.time()
    downstream_outputs[start_idx] = []
    # for idx in tqdm(range(5001,len(val_ds))):
    idx = start_idx
    failures = 0
    print(f"from idx = {idx} to {end_idx}")
    while idx < end_idx:
        try:
            out = get_output(val_ds[idx], additional = ". Do not output any additional text. and give proper parseable json format when asked for it")
            downstream_outputs[start_idx].append((idx, out))
            idx += 1
            failures = 0 #resetting failures.
            print("success for ", idx, " rate = ", (time.time() - start_time)/(idx), end = "          \r")
        except Exception as e:
            print("failed at ", idx, " because\n", e)
            failures += 1
            if(failures >= 6):
                downstream_outputs[start_idx].append((idx, "")) 
                print(f"IDX {idx} failed to calculate, setting to ''")
                failures = 0
                idx += 1
            else:
                time.sleep(4)
                continue    #sleep and then redo the same index.
        if(idx % 1000 == 0):
            with open(outname + f"_downstream_{start_idx}_{end_idx}.pkl", "wb") as f:
                pickle.dump(downstream_outputs[start_idx], f) #saving every 1000 iterations incase something goes wrong and generation has to stop in the middle.
    with open(outname + f"_downstream_{start_idx}_{end_idx}.pkl", "wb") as f:
        pickle.dump(downstream_outputs[start_idx], f)
    print(f"total time taken for {start_idx} to {end_idx} is {time.time() - start_time}")
    return downstream_outputs


In [8]:
all_outputs = []
partitions = 4 
overall_len = len(val_ds)
indices = []
for i in range(partitions+1):
    indices.append((i * overall_len)//partitions)

print(indices)
    
    

[0, 3085, 6171, 9256, 12342]


In [9]:
threads = [0]*partitions
# downstream_outputs = {}
for i in range(partitions):
    threads[i] = Thread(target = get_downstream_outputs_range, args = (indices[i], indices[i+1]))
for i in range(partitions):
    threads[i].start()
# for i in range(partitions):
    # threads[i].join()

from idx = 0 to 3085
from idx = 3085 to 6171
from idx = 6171 to 9256
from idx = 9256 to 12342
failed at  6210  because=  0.015695978170153738            
 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-429Lb6AirfSQULQ31VuPqF03 on tokens per min (TPM): Limit 30000, Used 29565, Requested 2573. Please try again in 4.276s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
failed at  6210  because 1.4867912327370993              
 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-429Lb6AirfSQULQ31VuPqF03 on tokens per min (TPM): Limit 30000, Used 28065, Requested 2573. Please try again in 1.276s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
failed at  9309  because 1.5168125527970335              
 Error code: 429 - {'error': {'messa

In [23]:
print(len(downstream_outputs[indices[3]]))

3086


In [9]:
for i in range(partitions):
    threads[i].join()
actual_outputs = []
for key in downstream_outputs:
    actual_outputs.extend(downstream_outputs[key])
actual_outputs = sorted(actual_outputs, key = lambda x: x[0])

failed at  7  becausete =  0.0007215210435167935           
 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-429Lb6AirfSQULQ31VuPqF03 on tokens per min (TPM): Limit 30000, Used 27947, Requested 2337. Please try again in 568ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
failed at  4938  because1.1729172468185425                
 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-429Lb6AirfSQULQ31VuPqF03 on tokens per min (TPM): Limit 30000, Used 28342, Requested 2143. Please try again in 970ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
failed at  8  becausete =  0.0012470555066652269          
 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-429Lb6AirfSQULQ31VuPqF03 on tokens per

KeyboardInterrupt: 

In [25]:
for i in range(partitions):
    threads[i].join()
actual_outputs = []
for key in downstream_outputs:
    actual_outputs.extend(downstream_outputs[key])
actual_outputs = sorted(actual_outputs, key = lambda x: x[0])

with open(outname + "_downstream_full.pkl", "wb") as f:
    pickle.dump(actual_outputs, f)

In [27]:
len(actual_outputs)

12342

## SIE Doping mof1 mof2 run

In [28]:
doping_outputs = []
exceptions = []
import time
# for idx in tqdm(range(len(doping_test))):
idx = 0
start = time.time()
failures = 0
while(idx < len(doping_test)):
    try:
        out = get_output(doping_test[idx], additional = ".Do not output any additional text")
        doping_outputs.append((idx, out))
        idx += 1
        print("at index ", idx, " at time: ", time.time() - start, "rate = ", (time.time() -start)/idx, "s/out", end = "   \r")
        failures = 0
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        exceptions.append((idx, e))
        failures += 1
        if(failures >= 6):
            failures = 0
            doping_outputs.append((idx,""))
            idx += 1
            print(f"setting {idx} to ''\n")
        time.sleep(1);
        # No updating the idx incase we encounter an error because it is likely a rate limit error.  
with open(outname + "_doping.pkl", "wb") as f:
    pickle.dump(doping_outputs, f)

In [29]:
mof1_outputs = []
exceptions = []
idx = 0;
start = time.time()
failures = 0
while(idx < len(mof1_test)):
    print("at index ", idx, " at time: ", time.time() - start, "rate = ", (idx/(time.time() -start)), end = "   \r")
    try:
        out = get_output(mof1_test[idx],additional = ".Do not output any additional text")
        mof1_outputs.append((idx, out))
        idx += 1
        failures = 0
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        failures += 1
        if(failures > 5):
            failures = 0
            print("setting ", idx, " to ''")
            mof1_outputs.append((idx, ''))
            idx += 1
        exceptions.append((idx, e))
        time.sleep(3)
with open(outname + "_mof1.pkl", "wb") as f:
    pickle.dump(mof1_outputs, f)

In [30]:
# with open("../llamat2_chat_train_2epochs_full_ex_mof1_test.pkl", "rb") as f:
#     check= pickle.load(f)

In [31]:
mof2_outputs = []
exceptions = []
idx = 0;
failures = 0
start = time.time()
while(idx < len(mof2_test)):
    print("at index ", idx, " at time: ", time.time() - start, "rate = ", (idx/(time.time() -start)), end = "   \r")
    try:
        out = get_output(mof2_test[idx],additional = "do not output any additional text")
        mof2_outputs.append((idx, out))
        idx += 1
        failures = 0
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        failures += 1
        if(failures > 5):
            failures = 0
            print("setting ", idx, " to ''")
            mof2_outputs.append((idx, ''))
            idx += 1
        exceptions.append((idx, e))
with open(outname + "_mof2.pkl", "wb") as f:
    pickle.dump(mof2_outputs, f)

## Discomat run

In [32]:
discomat_outputs = []
idx = 0;
start = time.time()
failures = 0
while(idx < len(discomat_test)):
    print("at index ", idx, " at time: ", time.time() - start, "rate = ", (idx/(time.time() -start)), end = "   \r")
    try:
        out = get_output(discomat_test[idx],additional = "do not output any additional text")
        discomat_outputs.append((idx, out))
        idx += 1
        failures = 0
    except Exception as e:
        print("failed at ", idx, " because\n", e)
        failures += 1
        if(failures > 5):
            failures = 0
            print("setting ", idx, " to ''")
            discomat_outputs.append((idx, ''))
            idx += 1
        exceptions.append((idx, e))
with open(outname + "_discomat.pkl", "wb") as f:
    pickle.dump(discomat_outputs, f)

In [ ]:
print("done")